In [1]:
import pandas as pd

clean_sampleset2000 = pd.read_csv('C:/Users/Devanand J/Desktop/Final/Yelp_dataset.csv')
restaurants_train2021 = pd.read_csv('C:/Users/Devanand J/Desktop/Final/SemEval.csv')

# Converting the 'overall_polarity' to 0 and 1 for compatibility 
clean_sampleset2000['overall_polarity'] = clean_sampleset2000['overall_polarity'].apply(lambda x: 0 if x == 1 else 1)


In [2]:
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    clean_sampleset2000['preprocessed_comments'], clean_sampleset2000['overall_polarity'], test_size=0.2, random_state=42)


In [3]:
from transformers import BertTokenizerFast

# We use the BERT here and tokenize the data
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts.tolist(), truncation=True, padding=True, max_length=512)


C:\Users\Devanand J\anaconda3\envs\gpt_cuda118\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, train_labels.tolist())
test_dataset = SentimentDataset(test_encodings, test_labels.tolist())


In [5]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Loading BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


C:\Users\Devanand J\anaconda3\envs\gpt_cuda118\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
import torch

# Training arguments 
training_args = TrainingArguments(
    num_train_epochs=3,              # number of training epochs
    per_device_train_batch_size=8,   # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    fp16=True if torch.cuda.is_available() else False,
)


In [7]:
trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

C:\Users\Devanand J\anaconda3\envs\gpt_cuda118\Lib\site-packages\accelerate\accelerator.py:447: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# Training the model
trainer.train()


Epoch,Training Loss,Validation Loss


In [ ]:
#Saving the model for later use
save_path = 'C:/Users/Devanand J/Desktop/Final/bert_finetuned_model'
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

In [ ]:
# Loading the model backfrom last trained checkpoint
#model = BertForSequenceClassification.from_pretrained(save_path)

#tokenizer = BertTokenizerFast.from_pretrained(save_path)

In [10]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

# Calculating accuracy and orinting the result
accuracy = accuracy_score(test_labels, preds)
print(f"Accuracy: {accuracy}")
report = classification_report(test_labels, preds, target_names=['negative', 'positive'])
print(report)


Evaluation results: {'eval_loss': 0.2909940779209137, 'eval_runtime': 412.3178, 'eval_samples_per_second': 0.97, 'eval_steps_per_second': 0.061, 'epoch': 3.0}
Accuracy: 0.925
              precision    recall  f1-score   support

    negative       0.92      0.93      0.93       201
    positive       0.93      0.92      0.92       199

    accuracy                           0.93       400
   macro avg       0.93      0.92      0.92       400
weighted avg       0.93      0.93      0.92       400



In [12]:
import pandas as pd
import numpy as np
from transformers import BertTokenizerFast, Trainer
import torch
from tqdm import tqdm
import time

# Predict on the entire unlabeled dataset with progress tracking
all_texts = clean_sampleset2000['preprocessed_comments'].tolist()
all_preds = []

batch_size = 16
num_batches = len(all_texts) // batch_size + (1 if len(all_texts) % batch_size != 0 else 0)

start_time = time.time()
for i in tqdm(range(num_batches), desc="Processing batches"):
    batch_texts = all_texts[i*batch_size:(i+1)*batch_size]
    batch_encodings = tokenizer(batch_texts, truncation=True, padding=True, max_length=512)
    batch_dataset = SentimentDataset(batch_encodings, [0]*len(batch_texts))  # Dummy labels

    batch_predictions = trainer.predict(batch_dataset)
    batch_preds = np.argmax(batch_predictions.predictions, axis=-1)
    all_preds.extend(batch_preds)

elapsed_time = time.time() - start_time
print(f"Processing completed in {elapsed_time:.2f} seconds")


clean_sampleset2000['predicted_sentiment'] = all_preds

# Saving the output with predictions
clean_sampleset2000.to_csv('C:/Users/Devanand J/Desktop/Final/BERT_overallpredictions.csv', index=False)

Processing batches:   0%|          | 0/125 [00:00<?, ?it/s]

Processing batches:   1%|          | 1/125 [00:09<20:18,  9.82s/it]

Processing batches:   2%|▏         | 2/125 [00:14<13:52,  6.77s/it]

Processing batches:   2%|▏         | 3/125 [00:26<18:30,  9.11s/it]

Processing batches:   3%|▎         | 4/125 [00:42<23:46, 11.79s/it]

...

Processing batches:  98%|█████████▊| 123/125 [29:35<00:46, 23.20s/it]

Processing batches:  99%|█████████▉| 124/125 [30:06<00:25, 25.42s/it]

Processing batches: 100%|██████████| 125/125 [30:34<00:00, 14.68s/it]


Processing completed in 1834.64 seconds


ModuleNotFoundError: No module named 'ace_tools'